In [2]:
import dicom
import inspect
import pylab
import numpy as np
from glob import glob
import os

import matplotlib
import matplotlib.pyplot as plt

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

from ml_utils import *
import math
from sys import stdout

/usr/local/lib/python3.5/dist-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)
Using TensorFlow backend.


In [4]:
benigns = np.load('benign_cropped.npy')
maligs = np.load('malig_cropped.npy')
nroi = np.load('nroi.npy')

In [5]:
benigns = preprocess(np.array(list(map(getFromSlice('roi'), flatten(benigns)))))
maligs = preprocess(np.array(list(map(getFromSlice('roi'), flatten(maligs)))))
nroi = preprocess(np.array(list(map(getFromSlice('nroi'), flatten(nroi)))))

In [6]:
np.random.shuffle(benigns)
np.random.shuffle(maligs)
np.random.shuffle(nroi)

In [7]:
nb = benigns.shape[0] // 3
nm = maligs.shape[0] // 3
nn = nroi.shape[0] // 3
b_t = benigns[:-nb]
b_v = benigns[-nb:]
m_t = maligs[:-nm]
m_v = maligs[-nm:]
n_t = nroi[:-nn]
n_v = nroi[-nn:]

In [8]:
data_gen = symmetric_data_generator([b_t, m_t, n_t],[np.array([0,1,1,0]),np.array([0,1,0,1]),np.array([1,0,0,0])], 96)

In [9]:
X, y = next(data_gen)

In [10]:
print(X.shape, y.shape)

(96, 80, 80, 1) (96, 4)


In [11]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(80, 80, 1)))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(Convolution2D(128, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Dense(4))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['precision', 'recall'])

In [12]:
imgGen = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True)

In [13]:
val_feeder = symmetric_data_generator([b_v, m_v, n_v], [np.array([0,1,1,0]),np.array([0,1,0,1]),np.array([1,0,0,0])], 192)

def evaluate(model):
    X, y = next(val_feeder)
    res = model.evaluate(X, y)
    print ('validation loss: %f\t prec: %f\trecall: %f' % tuple(res))

In [ ]:
nb_epoch = 30
nb_batch = 50

for e in range(nb_epoch):
    print ('Epoch %d' % e)
    mean_loss = np.array([0, 0, 0])
    for i in range(nb_batch):
        xx, yy = next(data_gen)
        X, y = next(imgGen.flow(xx, yy, batch_size=96))
        loss = np.array(model.train_on_batch(X, y))
        mean_loss = mean_loss + loss
        stdout.write('\rbatch - %d ## train loss: %f\t prec: %f\t recall: %f' % tuple([i] + list(mean_loss / (i + 1))))
    print("")
    evaluate(model)

Epoch 0
batch - 49 ## train loss: 0.015460	 prec: 0.994966	 recall: 0.995308
192/192 [==============================] - 1s     
validation loss: 0.486262	 prec: 0.919991	recall: 0.919991
Epoch 1
batch - 49 ## train loss: 0.009040	 prec: 0.996775	 recall: 0.997451
192/192 [==============================] - 1s     
validation loss: 0.245036	 prec: 0.969388	recall: 0.967731
Epoch 2
batch - 49 ## train loss: 0.014666	 prec: 0.995951	 recall: 0.995274
192/192 [==============================] - 1s    

In [33]:
evaluate(model)

192/192 [==============================] - 1s     
validation loss: 0.335467	 prec: 0.952507	recall: 0.952431


In [3]:
model2 = load_model('../models/model-babystep3-composite-tertiary-classification.h5')

In [84]:
do.get_config()

{'name': 'dropout_1', 'p': 0.1, 'trainable': False}